In [ ]:
from difflib import SequenceMatcher
import json
import pickle
import pandas as pd
from tqdm import tqdm

indexPath = "E:\Technologie\Master\Webm\subtitles_all.pkl"
movies_json = "data/movies.json"

output_file = "E:\Technologie\Master\Webm\subtitles_all_linked.pkl"

SLOW = False

In [ ]:
# load the dataframe from the pickle file
df = pd.read_pickle(indexPath)

In [ ]:
# load the json file
with open(movies_json,encoding="utf8") as json_file:
    data = json.load(json_file)

In [ ]:
# reduce the sizes for test purposes
movies = data[:1000]
df_sub_index = df[:]

In [ ]:
# delete the rows when there is no "title" key in the data json file
for i in range(len(movies)):
    if "uuid" not in movies[i] or 'title' not in movies[i] or "year" not in movies[i]:
        movies[i] = None

# delete the empty rows
movies = list(filter(None, movies))

In [ ]:
df_movies = pd.DataFrame(movies)

In [ ]:
# keep only columns we need
df_movies = df_movies[["uuid",'title', 'year']]

In [ ]:
# see the data structures
print(df_movies.columns)
print(df_sub_index.columns)

### Rapid method

In [ ]:
# Create a temporary column in both dataframes for merging
df_movies['key'] = df_movies['title'].str.lower() + df_movies['year'].astype(str)
df_sub_index['key'] = df_sub_index['MovieName'].str.lower() + df_sub_index['MovieYear'].astype(str)

# Merge the two dataframes on the 'key' column
df_merged = pd.merge(df_sub_index, df_movies, on='key', how='left')

# Drop the temporary 'key' columns
df_movies.drop(columns=['key'], inplace=True)
df_sub_index.drop(columns=['key'], inplace=True)
df_merged.drop(columns=['key'], inplace=True)

### Slow method

In [ ]:
SLOW = True

if SLOW == True:

    # function that matches an int only to another int if they are within a certain range
    def matchYear(year,cYear, yearDelta=1):
        if abs(year - cYear) > yearDelta:
            return False
        else:
            return True

    # function that deternmines the similarity between two movies titles and years
    def match_movies(title, year, title2, year2, threshold=0.90,yearDelta=1):
        if SequenceMatcher(None, title, title2).ratio() > threshold and matchYear(int(year),int(year2),yearDelta):
            return True
        else:
            return False

    # add a column to the dataframe that contains the movie uuid
    df_sub_index['movie_uuid'] = ""

    matches = []
    length = len(df_movies)

    for i,movie in tqdm(df_movies.iterrows(),total=length):
        for index, row in df_sub_index.iterrows():
            if match_movies(movie["title"],movie["year"],row["MovieName"],row["MovieYear"],threshold=0.9,yearDelta=1):
                df_sub_index.loc[index,'movie_uuid'] = movie["uuid"]
                break

### Results

In [ ]:
#print the rows that have a movie uuid
# df_sub_index[df_sub_index['movie_uuid'] != ""]

In [ ]:


# print the rows there movie uuid is not NaN
df_out = df_merged[df_merged['uuid'].notna()].drop(columns=["title","year"])
df_out

In [ ]:
# save the dataframe to a pickle file
df_out.to_pickle(output_file)